# TDEP

In [ ]:
# 安装 Conda 支持，一个 .ipynb 只需要执行一次即可
!pip install -q condacolab
import condacolab
condacolab.install()
# 安装完成后，会自动重启。

In [ ]:
!conda --version
!conda info --envs

In [ ]:
# 挂载 Google Drive，每次登录都要重新挂载
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

## TDEP 环境搭建

In [ ]:
# 本单元格大概耗时 4 分钟左右，请耐心等待

from IPython.display import clear_output as cls

# 创建 tdep 虚拟环境并安装 TDEP 依赖
!conda create -n tdep python=3.10 -y
# !conda env remove -n tdep -y # 如果需要，可以使用该命令删除环境
!conda run -n tdep conda install -c conda-forge gfortran openmpi scalapack fftw hdf5 h5py -y

# 安装 TDEP
!export PATH=/usr/local/envs/tdep/bin:$PATH
!rm -rf tdep
!git clone https://github.com/tdep-developers/tdep.git
%cd tdep
!cp ./examples/build/important_settings.conda ./important_settings
!sed -i "s|PREFIX=/path/to/your/conda/environment|PREFIX=/usr/local/envs/tdep|" ./important_settings # 使用 !conda info --envs 查看 PREFIX
!conda run -n tdep bash build_things.sh --nthreads_make 4

# 创建一个自定义的 .bashrc 文件用于加载环境变量
bashrc_content = """
export PATH=/usr/local/envs/tdep/bin:$PATH
MANPATH=$MANPATH:/content/tdep/man
PATH=$PATH:/content/tdep/bin
TDEP_BIN_DIR=/content/tdep/bin
alias gnuplot='gnuplot -persist'
"""
with open('./env_var.sh', 'w') as f:
    f.write(bashrc_content)
!chmod 777 /content/drive/MyDrive/Colab_Notebooks/TDEP/env_var.sh

# 重新加载 env_var.sh 文件，即可在新的单元格中使设置生效
# !bash /content/tdep/env_var.sh

cls()
!echo "Finished."

In [ ]:
# 可以选择导出环境配置以便在其它 notebook 使用
# !conda env export -n tdep > /content/drive/MyDrive/tdep_environment.yml

# 然后在其它 colab 中使用导出的 YAML 文件重新创建环境
# !conda env create -f /content/drive/MyDrive/tdep_environment.yml

In [ ]:
'''
此节为固定开头
'''
from IPython.display import clear_output as cls
%cd /content


'''
添加新用户和 home 目录，并赋予读、写、执行权限。执行一次即可。
'''
!id guest1222 || useradd guest1222
!mkdir -p /home/guest1222
!chmod 777 /home -R
!chmod 777 /tmp -R
!chmod 777 /var -R 2>/dev/null

'''
删除用户
'''
# !userdel -r guest1222 2>/dev/null # 删除用户 guest1222 以及该用户的 home 目录。
# !rm -rf /home/guest1222 2>/dev/null # 使用 rm -rf 强制删除目录及其下的所有内容。

'''
此节为固定结尾
'''
# 回到默认目录，以免影响临时添加代码块进行操作时所在的目录
%cd /content
# 以 python 函数方式运行 cls 清除输出（不想清除请注释此句）
cls()
# 提示结束
!echo "Finished."

## TDEP 教程

In [ ]:
!if [ -d /home/guest1222/workdir/ ]; then rm -rf /home/guest1222/workdir/; fi
!mkdir -p /home/guest1222/workdir/
!cp -r /content/drive/MyDrive/Colab_Notebooks/TDEP /home/guest1222/workdir/
!ls /home/guest1222/workdir/TDEP/*

In [ ]:
from IPython.display import clear_output as cls
%cd /content

!chmod 777 /home -R
!chmod 777 /tmp -R
!chmod 777 /var -R 2>/dev/null

'''
输入命令！
'''
!>  /tmp/cmd.txt
!>> /tmp/cmd.txt echo 'cd /home/guest1222/workdir/TDEP/infiles_T300K'
!>> /tmp/cmd.txt echo 'ls'
!>> /tmp/cmd.txt echo 'bash /content/tdep/env_var.sh'
!>> /tmp/cmd.txt echo 'conda run -n tdep /content/tdep/bin/extract_forceconstants -rc2 12 -rc3 8'
!>> /tmp/cmd.txt echo 'ls'

'''
运行！
'''
!<  /tmp/cmd.txt su guest1222

# cls()
!echo "Terminated."

In [ ]:
from IPython.display import clear_output as cls
%cd /content

!chmod 777 /home -R
!chmod 777 /tmp -R
!chmod 777 /var -R 2>/dev/null

'''
输入命令！
'''
!>  /tmp/cmd.txt
!>> /tmp/cmd.txt echo 'cd /home/guest1222/workdir/TDEP/infiles_T300K'
!>> /tmp/cmd.txt echo 'bash /content/tdep/env_var.sh'
!>> /tmp/cmd.txt echo 'cp outfile.forceconstant infile.forceconstant'
!>> /tmp/cmd.txt echo 'cp outfile.forceconstant_thirdorder infile.forceconstant_thirdorder'
# !>> /tmp/cmd.txt echo 'conda run -n tdep /content/tdep/bin/phonon_dispersion_relations --help'
!>> /tmp/cmd.txt echo 'conda run -n tdep /content/tdep/bin/phonon_dispersion_relations'
# !>> /tmp/cmd.txt echo 'conda run -n tdep /content/tdep/bin/lineshape --help'
!>> /tmp/cmd.txt echo 'conda run -n tdep /content/tdep/bin/lineshape --path -qg 6 6 6 -ne 600 --temperature 300'
!>> /tmp/cmd.txt echo 'ls'

'''
运行！
'''
!<  /tmp/cmd.txt su guest1222

# cls()
!echo "Terminated."

In [ ]:
from matplotlib.colors import LogNorm

import h5py as h5
import numpy as np
import matplotlib.pyplot as plt

f = h5.File("/home/guest1222/workdir/TDEP/infiles_T300K/outfile.phonon_spectral_function.hdf5", "r")
print(f.keys())

x = np.array(f.get("q_values"))
y = np.array(f.get("energy_values"))
gz = np.array(f.get("spectral_function"))
xt = np.array(f.get('q_ticks'))
xl = f.attrs.get('q_tick_labels').split()
gz=gz+1E-2
gx, gy = np.meshgrid(x,y)

plt.pcolormesh(gx, gy, gz, norm=LogNorm(vmin=gz.min(), vmax=gz.max()), cmap='afmhot')
plt.axis([x.min(), x.max(), y.min(), 5])
plt.xlabel("KPATH")
plt.ylabel("Frequency (THz)")
plt.xticks(xt, xl)


plt.savefig("phon.png", bbox_inches='tight')